In [1]:
# !pip install -r ../input/fathomnet-2025/requirements.txt

In [2]:
from PIL import Image
import pandas as pd
import torch

In [3]:
tree_mapping = {
    "Vesicomyidae": ["Animalia", "Mollusca", "Bivalvia", "Veneroida", "Vesicomyidae"],
    "Sebastolobus": ["Animalia", "Chordata", "Actinopterygii", "Scorpaeniformes", "Sebastidae", "Sebastolobus"],
    "Apostichopus leukothele": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Stichopodidae", "Apostichopus leukothele"],
    "Scotoplanes": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Scotoplanes"],
    "Keratoisis": ["Animalia", "Porifera", "Demospongiae", "Verongimorpha", "Keratoisidae", "Keratoisis"],
    "Munnopsidae": ["Animalia", "Arthropoda", "Malacostraca", "Isopoda", "Munnopsidae"],
    "Scotoplanes globosa": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Scotoplanes globosa"],
    "Mediaster aequalis": ["Animalia", "Echinodermata", "Asteroidea", "Forcipulatida", "Asteriidae", "Mediaster aequalis"],
    "Asbestopluma monticola": ["Animalia", "Porifera", "Demospongiae", "Verongimorpha", "Asbestopluma", "Asbestopluma monticola"],
    "Acanthascinae": ["Animalia", "Echinodermata", "Asteroidea", "Valvatida", "Acanthasteridae", "Acanthascinae"],
    "Florometra serratissima": ["Animalia", "Echinodermata", "Crinoidea", "Comatulida", "Antedonidae", "Florometra serratissima"],
    "Psathyrometra fragilis": ["Animalia", "Echinodermata", "Crinoidea", "Comatulida", "Antedonidae", "Psathyrometra fragilis"],
    "Zoantharia": ["Animalia", "Cnidaria", "Anthozoa", "Zoantharia"],
    "Gersemia juliepackardae": ["Animalia", "Cnidaria", "Anthozoa", "Alcyonacea", "Isididae", "Gersemia juliepackardae"],
    "Pandalus platyceros": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Pandalidae", "Pandalus platyceros"],
    "Isidella tentaculum": ["Animalia", "Cnidaria", "Anthozoa", "Alcyonacea", "Isididae", "Isidella tentaculum"],
    "Holothuroidea": ["Animalia", "Echinodermata", "Holothuroidea"],
    "Paragorgia arborea": ["Animalia", "Cnidaria", "Anthozoa", "Gorgonacea", "Isididae", "Paragorgia arborea"],
    "Heterochone calyx": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Heterochone calyx"],
    "Paragorgia": ["Animalia", "Cnidaria", "Anthozoa", "Gorgonacea", "Isididae", "Paragorgia"],
    "Terebellidae": ["Animalia", "Annelida", "Polychaeta", "Terebellida", "Terebellidae"],
    "Ophiuroidea": ["Animalia", "Echinodermata", "Ophiuroidea"],
    "Peniagone": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Peniagone"],
    "Actiniaria": ["Animalia", "Cnidaria", "Anthozoa", "Actiniaria"],
    "Ceriantharia": ["Animalia", "Cnidaria", "Anthozoa", "Ceriantharia"],
    "Isididae": ["Animalia", "Cnidaria", "Anthozoa", "Gorgonacea", "Isididae"],
    "Rathbunaster californicus": ["Animalia", "Echinodermata", "Asteroidea", "Forcipulatida", "Asteriidae", "Rathbunaster californicus"],
    "Paelopatides confundens": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Paelopatides confundens"],
    "Abyssocucumis abyssorum": ["Animalia", "Echinodermata", "Holothuroidea", "Aspidochirotida", "Stichopodidae", "Abyssocucumis abyssorum"],
    "Strongylocentrotus fragilis": ["Animalia", "Echinodermata", "Echinoidea", "Clypeasteroida", "Strongylocentrotidae", "Strongylocentrotus fragilis"],
    "Porifera": ["Animalia", "Porifera"],
    "Psolus squamatus": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Stichopodidae", "Psolus squamatus"],
    "Liponema brevicorne": ["Animalia", "Echinodermata", "Holothuroidea", "Dendrochirotida", "Elasipodidae", "Liponema brevicorne"],
    "Farrea": ["Animalia", "Porifera", "Demospongiae", "Haplosclerida", "Mycalidae", "Farrea"],
    "Caridea": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Caridea"],
    "Hexactinellida": ["Animalia", "Porifera", "Hexactinellida"],
    "Actinernus": ["Animalia", "Echinodermata", "Asteroidea", "Forcipulatida", "Asteriidae", "Actinernus"],
    "Pannychia moseleyi": ["Animalia", "Porifera", "Demospongiae", "Haplosclerida", "Mycalidae", "Pannychia moseleyi"],
    "Gastropoda": ["Animalia", "Mollusca", "Gastropoda"],
    "Benthocodon pedunculata": ["Animalia", "Cnidaria", "Hydrozoa", "Hydroidolina", "Leptomedusae", "Benthocodon pedunculata"],
    "Octopus rubescens": ["Animalia", "Mollusca", "Cephalopoda", "Octopodiformes", "Octopodidae", "Octopus rubescens"],
    "Microstomus pacificus": ["Animalia", "Chordata", "Actinopterygii", "Pleuronectiformes", "Paralichthyidae", "Microstomus pacificus"],
    "Heteropolypus ritteri": ["Animalia", "Cnidaria", "Hydrozoa", "Hydroidolina", "Hydridae", "Heteropolypus ritteri"],
    "Parastenella": ["Animalia", "Cnidaria", "Hydrozoa", "Hydroidolina", "Hydridae", "Parastenella"],
    "Staurocalyptus": ["Animalia", "Cnidaria", "Hydrozoa", "Hydroidolina", "Staurocalyptidae", "Staurocalyptus"],
    "Hormathiidae": ["Animalia", "Cnidaria", "Anthozoa", "Actiniaria", "Hormathiidae"],
    "Scleractinia": ["Animalia", "Cnidaria", "Anthozoa", "Scleractinia"],
    "Asbestopluma": ["Animalia", "Porifera", "Demospongiae", "Verongimorpha", "Asbestopluma"],
    "Isosicyonis": ["Animalia", "Porifera", "Demospongiae", "Haplosclerida", "Mycalidae", "Isosicyonis"],
    "Umbellula": ["Animalia", "Cnidaria", "Anthozoa", "Gorgonacea", "Umbellulidae", "Umbellula"],
    "Sebastes": ["Animalia", "Chordata", "Actinopterygii", "Scorpaeniformes", "Sebastidae", "Sebastes"],
    "Metridium farcimen": ["Animalia", "Cnidaria", "Anthozoa", "Actiniaria", "Metridiidae", "Metridium farcimen"],
    "Merluccius productus": ["Animalia", "Chordata", "Actinopterygii", "Gadiformes", "Gadidae", "Merluccius productus"],
    "Funiculina": ["Animalia", "Echinodermata", "Crinoidea", "Comatulida", "Funiculinidae", "Funiculina"],
    "Brisingida": ["Animalia", "Echinodermata", "Asteroidea", "Brisingida"],
    "Pyrosoma atlanticum": ["Animalia", "Chordata", "Tunicates", "Pyrosomida", "Pyrosomatidae", "Pyrosoma atlanticum"],
    "Acanthoptilum": ["Animalia", "Chordata", "Actinopterygii", "Perciformes", "Acanthoptilidae", "Acanthoptilum"],
    "Actinopterygii": ["Animalia", "Chordata", "Actinopterygii"],
    "Chorilia longipes": ["Animalia", "Cnidaria", "Anthozoa", "Alcyonacea", "Isididae", "Chorilia longipes"],
    "Paralomis multispina": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Lithodidae", "Paralomis multispina"],
    "Sebastes diploproa": ["Animalia", "Chordata", "Actinopterygii", "Scorpaeniformes", "Sebastidae", "Sebastes diploproa"],
    "Mycale": ["Animalia", "Porifera", "Demospongiae", "Haplosclerida", "Mycalidae", "Mycale"],
    "Lithodidae": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Lithodidae"],
    "Ophiacanthidae": ["Animalia", "Echinodermata", "Ophiuroidea", "Ophiacanthidae"],
    "Corallimorphus pilatus": ["Animalia", "Cnidaria", "Anthozoa", "Corallimorpharia", "Corallimorphidae", "Corallimorphus pilatus"],
    "Ptychogastria polaris": ["Animalia", "Cnidaria", "Hydrozoa", "Hydroidea", "Ptychogastriidae", "Ptychogastria polaris"],
    "Amphipoda": ["Animalia", "Arthropoda", "Malacostraca", "Amphipoda"],
    "Heterocarpus": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Heterocarpoidea", "Heterocarpus"],
    "Pennatula phosphorea": ["Animalia", "Cnidaria", "Anthozoa", "Pennatulacea", "Pennatulidae", "Pennatula phosphorea"],
    "Chionoecetes tanneri": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Oregoniidae", "Chionoecetes tanneri"],
    "Hippasteria": ["Animalia", "Echinodermata", "Asteroidea", "Valvatida", "Asteriidae", "Hippasteria"],
    "Munidopsis": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Munidopsidae", "Munidopsis"],
    "Serpulidae": ["Animalia", "Annelida", "Polychaeta", "Canalipalpata", "Serpulidae"],
    "Delectopecten": ["Animalia", "Mollusca", "Bivalvia", "Pectinida", "Pectinidae", "Delectopecten"],
    "Crinoidea": ["Animalia", "Echinodermata", "Crinoidea"],
    "Tunicata": ["Animalia", "Chordata", "Tunicata"],
    "Asteroidea": ["Animalia", "Echinodermata", "Asteroidea"],
    "Pandalus amplus": ["Animalia", "Arthropoda", "Malacostraca", "Decapoda", "Pandalidae", "Pandalus amplus"],
    "Elpidia": ["Animalia", "Echinodermata", "Holothuroidea", "Aspidochirotida", "Stichopodidae", "Elpidia"]
}
import json
with open('/kaggle/input/fathomnet-2025/dataset_train.json', 'r') as f:
    data = json.load(f)
id_to_name = {i: category['name'] for i, category in enumerate(data['categories'])}
keys = list(tree_mapping.keys())
k = 0
for i in range(len(keys)):
    k = max(len(tree_mapping[keys[i]]), k)
for i in range(len(keys)):
    tree_mapping[keys[i]].append("spl_stp")
    tree_mapping[keys[i]] += ["-"] * (7 - len(tree_mapping[keys[i]]))

columns = ["Kingdom", "Phylum", "Class", "Order", "Family", "Genus_Species", "spl"]
df = pd.DataFrame.from_dict(tree_mapping, orient='index', columns=columns)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Taxon'}, inplace=True)
unique_elements = pd.unique(df.values.ravel())
onehot = dict(pd.get_dummies(unique_elements).astype(int))
for i in range(len(list(tree_mapping.keys()))):
    for j in range(7):
        tree_mapping[list(tree_mapping.keys())[i]][j] = onehot[tree_mapping[list(tree_mapping.keys())[i]][j]]
labels_idx = [int(torch.tensor(onehot[i]).argmax()) for i in id_to_name.values()]

In [4]:
from torchvision import transforms

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

test_transforms = val_transforms  # identical to val

----------------**DataSet Classes**-----------------

In [5]:
# from torch.utils.data import Dataset
# import pandas as pd
# from PIL import Image
# import torch
# import ast


# class FathomNetDataset_tkn(Dataset):
#     def __init__(self, csv_path, transformation=None, is_test=False, tree_map=tree_mapping):
#         self.data = pd.read_csv(csv_path)
#         self.transform = transformation
#         self.is_test = is_test
#         self.image_paths = self.data["path"].tolist()
#         self.num_class = len(list(tree_map.keys()))

#         if not is_test:
#             self.labels = self.data["label"]
#             # one_hot = pd.get_dummies(self.labels)
#             self.updated_labels = self.labels.map(lambda x: tree_mapping[x])
#             # self.class_mapping = dict(enumerate(one_hot.columns.tolist()))
#             # self.class_mapping = tree_map
#         else:
#             self.labels = None
#             self.label_encoded = None
            
#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img = Image.open(self.image_paths[idx]).convert('RGB')
#         img_trans = self.transform(img) if self.transform else img
    
#         if self.is_test:
#             return img_trans, self.image_paths[idx]
#         else:
#             label = self.updated_labels[idx]
#             label_tensor = torch.tensor(label)
#             return img_trans, label_tensor

# class FathomNetDataset_indep(Dataset):

#     def __init__(self, csv_path, transform=None, label_encoder=None, is_test=False):
#         self.data = pd.read_csv(csv_path)
#         self.transform = transform
#         self.is_test = is_test

#         self.image_paths = self.data["path"].tolist()

#         if not is_test:
#             self.labels = self.data["label"].tolist()
#             # Use provided label encoder or fit one
#             if label_encoder is None:
#                 self.label_encoder = LabelEncoder()
#                 self.label_ids = self.label_encoder.fit_transform(self.labels)
#             else:
#                 self.label_encoder = label_encoder
#                 self.label_ids = self.label_encoder.transform(self.labels)
#         else:
#             self.labels = None
#             self.label_ids = None

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = Image.open(self.image_paths[idx]).convert("RGB")
#         if self.transform:
#             image = self.transform(image)

#         if self.is_test:
#             return image, self.image_paths[idx]
#         else:
#             label = self.label_ids[idx]
#             return image, label

In [6]:
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image

class FathomNetDataset_tkn(Dataset):
    def __init__(self, csv_path, transformation=None, is_test=False):
        self.data = pd.read_csv(csv_path)
        self.transform = transformation
        self.is_test = is_test
        self.image_paths = self.data["path"].tolist()

        if not self.is_test:
            self.labels = self.data["label"].tolist()
        else:
            self.labels = None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)

        if self.is_test:
            return img, self.image_paths[idx]
        else:
            label = self.labels[idx]
            return img, label


class FathomNetDataset_indep(Dataset):
    def __init__(self, csv_path, transform=None, is_test=False, label_encoder=None):
        self.data = pd.read_csv(csv_path)
        self.transform = transform
        self.is_test = is_test
        self.image_paths = self.data["path"].tolist()

        if not self.is_test:
            self.labels = self.data["label"].tolist()
        else:
            self.labels = None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, self.image_paths[idx]
        else:
            label = self.labels[idx]
            return image, label


----------------------**Model Classes**--------------------------

In [7]:
# N-ary Tree LSTM

from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.models import EfficientNet_V2_M_Weights
from PIL import Image
import pandas as pd
import timm

class ChildSumTreeLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.child_num = max_len  # max number of children

        # Input projections
        self.W_i = nn.Linear(input_size, hidden_size)
        self.W_f = nn.Linear(input_size, hidden_size)
        self.W_o = nn.Linear(input_size, hidden_size)
        self.W_u = nn.Linear(input_size, hidden_size)

        # Distinct hidden projections per child (indexed by position)
        self.U_i = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_f = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_o = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_u = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])

        # Bias terms
        self.b_i = nn.Parameter(torch.zeros(hidden_size))
        self.b_f = nn.Parameter(torch.zeros(hidden_size))
        self.b_o = nn.Parameter(torch.zeros(hidden_size))
        self.b_u = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x, child_h, child_c):
        # Pad child_h and child_c with zeros up to self.child_num
        batch_size, device = x.size(0), x.device
        zero_h = torch.zeros(batch_size, self.hidden_size, device=device)
        zero_c = torch.zeros(batch_size, self.hidden_size, device=device)

        # Fill missing child states
        child_h = child_h + [zero_h] * (self.child_num - len(child_h))
        child_c = child_c + [zero_c] * (self.child_num - len(child_c))

        # Sum over active children only (i.e., up to original len)
        active_len = len(child_h) - (self.child_num - len(child_h))

        # Gates
        i_input = self.W_i(x) + sum(self.U_i[k](child_h[k]) for k in range(active_len)) + self.b_i
        o_input = self.W_o(x) + sum(self.U_o[k](child_h[k]) for k in range(active_len)) + self.b_o
        u_input = self.W_u(x) + sum(self.U_u[k](child_h[k]) for k in range(active_len)) + self.b_u

        i = torch.sigmoid(i_input)
        o = torch.sigmoid(o_input)
        u = torch.tanh(u_input)

        # Forget gates and new cell state
        c = i * u
        for k in range(active_len):
            f_k = torch.sigmoid(self.W_f(x) + self.U_f[k](child_h[k]) + self.b_f)
            c += f_k * child_c[k]

        h = o * torch.tanh(c)
        return h, c


class FathomnetClassTree(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, stop_tkn_idx, pad_tkn_idx, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.stop_tkn_idx = stop_tkn_idx
        self.pad_tkn_idx = pad_tkn_idx
        self.max_len = max_len

        # EfficientNet feature extractor
        base = models.efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(
            base.features,
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
        )

        # Project image features to LSTM hidden size
        self.img_proj = nn.Linear(input_dim, hidden_size)

        # TreeLSTM cell (shared across time)
        self.tree_cell = ChildSumTreeLSTMCell(input_size=input_dim, hidden_size=hidden_size, max_len=max_len)

        # Final classifier head
        self.fc = nn.Linear(hidden_size, vocab_size)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        batch_size, device = x.size(0), x.device

        # 1. Extract image features
        feats = self.feature_extractor(x)  # [B, input_dim]
        img_h = torch.tanh(self.img_proj(feats))  # [B, H]
        img_c = torch.zeros_like(img_h)

        # 2. Init state
        hidden_states = [img_h]
        cell_states = [img_c]
        logits_list = []

        done = torch.zeros(batch_size, dtype=torch.bool, device=device)

        # 3. Decode for max_len steps
        for t in range(self.max_len):
            h_t, c_t = self.tree_cell(feats, hidden_states, cell_states)
            hidden_states.append(h_t)
            cell_states.append(c_t)

            logits = self.fc(h_t)  # [B, V]
            logits_list.append(logits.unsqueeze(1))

            preds = torch.argmax(logits, dim=1)
            done |= (preds == self.stop_tkn_idx)
            if done.all():
                break

        # 4. Padding if needed
        output = torch.cat(logits_list, dim=1)
        pad_len = self.max_len - output.size(1)
        if pad_len > 0:
            pad_logits = torch.zeros(batch_size, pad_len, self.vocab_size, device=device)
            pad_logits[:, :, self.pad_tkn_idx] = 1  # Add one-hot pad probs
            output = torch.cat([output, pad_logits], dim=1)

        return output  # [B, max_len, vocab_size]

    def criterion(self, x, y):
        logits = self.forward(x)                   # [B, 7, V]
        y = y.float()                              # [B, 7, V]
        logits_flat = logits.view(-1, self.vocab_size)
        targets_flat = y.view(-1, self.vocab_size)
        return self.loss(logits_flat, targets_flat)





class FathomnetClassLSTM(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, stop_tkn_idx, pad_tkn_idx, max_len):
        super().__init__()
        # Feature extractor
        self.model = models.efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(
            self.model.features,
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )

        # GRU decoder (changed from LSTM to GRU)
        self.lstm = nn.LSTM(input_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

        # Direct index assignment from known tensors
        self.stop_tkn_idx = stop_tkn_idx
        self.pad_tkn = pad_tkn_idx
        self.max_len = max_len
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.lambda_path = 0.3

        # Loss function ignoring padding tokens
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        batch_size, device = x.size(0), x.device
        feats = self.feature_extractor(x)              # [B, input_dim]
        input_seq = feats.unsqueeze(1)                 # [B, 1, input_dim]

        hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)
        cell   = torch.zeros(1, batch_size, self.hidden_size, device=device)

        logits_list = []                               # [B, t, V]
        done = [False] * batch_size

        for t in range(self.max_len):
            out, (hidden, cell) = self.lstm(input_seq, (hidden, cell))  # [B,1,H]
            logits = self.fc(out.squeeze(1))                             # [B, V]
            logits_list.append(logits.unsqueeze(1))                      # [B,1,V]

            preds = torch.argmax(logits, dim=1)                          # [B]
            for i in range(batch_size):
                if not done[i] and preds[i].item() == 5:
                    done[i] = True
            if all(done):
                break

            input_seq = feats.unsqueeze(1)

        logits_tensor = torch.cat(logits_list, dim=1)  # [B, T_pred, V]; Here T-pred is varible <= 7
        # pad to max_len
        if logits_tensor.size(1) < self.max_len:
            pad_size = self.max_len - logits_tensor.size(1)
            pad_logits = torch.zeros(batch_size, pad_size, self.vocab_size, device=device)
            pad_logits[:, :, 6] = 1
            logits_tensor = torch.cat([logits_tensor, pad_logits], dim=1)

        return logits_tensor   #[B, 7, V]





class FathomnetClass(nn.Module):
    def __init__(self, num_classes):
        super(FathomnetClass, self).__init__()
        
        # Load pretrained EfficientNetV2-M backbone
        self.backbone = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        
        # Replace the classifier head
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Sequential(
            nn.Linear(in_features, 48),
            nn.ReLU(inplace=True),
            nn.Dropout(0.15),
            nn.Linear(48, num_classes)
        )
        # Loss function
        self.loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    def forward(self, x):
        """Forward pass."""
        return self.backbone(x)

    def compute_loss(self, x, labels):
        """Returns loss and logits."""
        logits = self.forward(x)
        loss = self.loss_fn(logits, labels)
        return loss, logits


class ChildSumTreeLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.child_num = max_len

        self.W_i = nn.Linear(input_size, hidden_size)
        self.W_f = nn.Linear(input_size, hidden_size)
        self.W_o = nn.Linear(input_size, hidden_size)
        self.W_u = nn.Linear(input_size, hidden_size)

        self.U_i = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_f = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_o = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_u = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])

        self.b_i = nn.Parameter(torch.zeros(hidden_size))
        self.b_f = nn.Parameter(torch.zeros(hidden_size))
        self.b_o = nn.Parameter(torch.zeros(hidden_size))
        self.b_u = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x, child_h, child_c):
        batch_size, device = x.size(0), x.device
        zero_h = torch.zeros(batch_size, self.hidden_size, device=device)
        zero_c = torch.zeros(batch_size, self.hidden_size, device=device)

        actual_len = len(child_h)
        child_h = child_h + [zero_h] * (self.child_num - actual_len)
        child_c = child_c + [zero_c] * (self.child_num - actual_len)

        i_input = self.W_i(x) + sum(self.U_i[k](child_h[k]) for k in range(actual_len)) + self.b_i
        o_input = self.W_o(x) + sum(self.U_o[k](child_h[k]) for k in range(actual_len)) + self.b_o
        u_input = self.W_u(x) + sum(self.U_u[k](child_h[k]) for k in range(actual_len)) + self.b_u

        i = torch.sigmoid(i_input)
        o = torch.sigmoid(o_input)
        u = torch.tanh(u_input)

        c = i * u
        for k in range(actual_len):
            f_k = torch.sigmoid(self.W_f(x) + self.U_f[k](child_h[k]) + self.b_f)
            c += f_k * child_c[k]

        h = o * torch.tanh(c)
        return h, c


class FathomnetClassResnet(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, stop_tkn_idx, pad_tkn_idx, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.stop_tkn_idx = stop_tkn_idx
        self.pad_tkn_idx = pad_tkn_idx
        self.max_len = max_len

        # EfficientNet backbone
        backbone = timm.create_model("resnest269e", pretrained=True, num_classes=0)  # num_classes=0 disables classifier
        self.feature_extractor = nn.Sequential(
            backbone,  # full model will just return features
            nn.Flatten()
        )
        feat_dim = backbone.num_features  # e.g., 1280 for B0

        self.img_proj = nn.Linear(feat_dim, hidden_size)

        # Token embedding
        self.token_emb = nn.Embedding(vocab_size, hidden_size)
        self.start_token = nn.Parameter(torch.randn(1, hidden_size))

        # TreeLSTM
        self.tree_cell = ChildSumTreeLSTMCell(input_size=hidden_size, hidden_size=hidden_size, max_len=max_len)

        # Classifier
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets=None):
        batch_size, device = x.size(0), x.device

        # 1. Encode image
        feats = self.feature_extractor(x)  # [B, feat_dim]
        img_h = torch.tanh(self.img_proj(feats))  # [B, H]
        img_c = torch.zeros_like(img_h)

        hidden_states = [img_h]
        cell_states = [img_c]
        logits_list = []

        done = torch.zeros(batch_size, dtype=torch.bool, device=device)

        current_token = self.start_token.expand(batch_size, -1)

        for t in range(self.max_len):
            h_t, c_t = self.tree_cell(current_token, hidden_states, cell_states)
            hidden_states.append(h_t)
            cell_states.append(c_t)

            logits = self.fc(h_t)  # [B, V]
            logits_list.append(logits.unsqueeze(1))

            if targets is not None:
                # Teacher forcing
                current_token = self.token_emb(targets[:, t])
            else:
                # Inference
                preds = torch.argmax(logits, dim=1)
                done |= (preds == self.stop_tkn_idx)
                current_token = self.token_emb(preds)
                if done.all():
                    break

        output = torch.cat(logits_list, dim=1)

        if targets is not None:
            loss = self.loss(output.view(-1, self.vocab_size), targets.view(-1))
            return output, loss

        # Pad if needed
        pad_len = self.max_len - output.size(1)
        if pad_len > 0:
            pad_logits = torch.zeros(batch_size, pad_len, self.vocab_size, device=device)
            pad_logits[:, :, self.pad_tkn_idx] = 1
            output = torch.cat([output, pad_logits], dim=1)

        return output  # [B, max_len, vocab_size]



# Swin Transformer Model

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import timm  # for ResNeST backbone
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # for progress bars


class ChildSumTreeLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.child_num = max_len

        self.W_i = nn.Linear(input_size, hidden_size)
        self.W_f = nn.Linear(input_size, hidden_size)
        self.W_o = nn.Linear(input_size, hidden_size)
        self.W_u = nn.Linear(input_size, hidden_size)

        self.U_i = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_f = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_o = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])
        self.U_u = nn.ModuleList([nn.Linear(hidden_size, hidden_size, bias=False) for _ in range(max_len)])

        self.b_i = nn.Parameter(torch.zeros(hidden_size))
        self.b_f = nn.Parameter(torch.zeros(hidden_size))
        self.b_o = nn.Parameter(torch.zeros(hidden_size))
        self.b_u = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x, child_h, child_c):
        batch_size, device = x.size(0), x.device
        zero_h = torch.zeros(batch_size, self.hidden_size, device=device)
        zero_c = torch.zeros(batch_size, self.hidden_size, device=device)

        actual_len = len(child_h)
        child_h = child_h + [zero_h] * (self.child_num - actual_len)
        child_c = child_c + [zero_c] * (self.child_num - actual_len)

        i_input = self.W_i(x) + sum(self.U_i[k](child_h[k]) for k in range(actual_len)) + self.b_i
        o_input = self.W_o(x) + sum(self.U_o[k](child_h[k]) for k in range(actual_len)) + self.b_o
        u_input = self.W_u(x) + sum(self.U_u[k](child_h[k]) for k in range(actual_len)) + self.b_u

        i = torch.sigmoid(i_input)
        o = torch.sigmoid(o_input)
        u = torch.tanh(u_input)

        c = i * u
        for k in range(actual_len):
            f_k = torch.sigmoid(self.W_f(x) + self.U_f[k](child_h[k]) + self.b_f)
            c += f_k * child_c[k]

        h = o * torch.tanh(c)
        return h, c


class FathomnetClassSwin(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, stop_tkn_idx, pad_tkn_idx, max_len):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.stop_tkn_idx = stop_tkn_idx
        self.pad_tkn_idx = pad_tkn_idx
        self.max_len = max_len

        # EfficientNet backbone
        backbone = timm.create_model("swin_large_patch4_window7_224", pretrained=True, num_classes=0)  # num_classes=0 disables classifier
        self.feature_extractor = nn.Sequential(
            backbone,  # full model will just return features
            nn.Flatten()
        )
        feat_dim = backbone.num_features    # [batch_size, 1536]

        self.img_proj = nn.Linear(feat_dim, hidden_size)

        # Token embedding
        self.token_emb = nn.Embedding(vocab_size, hidden_size)
        self.start_token = nn.Parameter(torch.randn(1, hidden_size))

        # TreeLSTM
        self.tree_cell = ChildSumTreeLSTMCell(input_size=hidden_size, hidden_size=hidden_size, max_len=max_len)

        # Classifier
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets=None):
        batch_size, device = x.size(0), x.device

        # 1. Encode image
        feats = self.feature_extractor(x)  # [B, feat_dim]
        img_h = torch.tanh(self.img_proj(feats))  # [B, H]
        img_c = torch.zeros_like(img_h)

        hidden_states = [img_h]
        cell_states = [img_c]
        logits_list = []

        done = torch.zeros(batch_size, dtype=torch.bool, device=device)

        current_token = self.start_token.expand(batch_size, -1)

        for t in range(self.max_len):
            h_t, c_t = self.tree_cell(current_token, hidden_states, cell_states)
            hidden_states.append(h_t)
            cell_states.append(c_t)

            logits = self.fc(h_t)  # [B, V]
            logits_list.append(logits.unsqueeze(1))

            if targets is not None:
                # Teacher forcing
                current_token = self.token_emb(targets[:, t])
            else:
                # Inference
                preds = torch.argmax(logits, dim=1)
                done |= (preds == self.stop_tkn_idx)
                current_token = self.token_emb(preds)
                if done.all():
                    break

        output = torch.cat(logits_list, dim=1)

        if targets is not None:
            loss = self.loss(output.view(-1, self.vocab_size), targets.view(-1))
            return output, loss

        # Pad if needed
        pad_len = self.max_len - output.size(1)
        if pad_len > 0:
            pad_logits = torch.zeros(batch_size, pad_len, self.vocab_size, device=device)
            pad_logits[:, :, self.pad_tkn_idx] = 1
            output = torch.cat([output, pad_logits], dim=1)

        return output  # [B, max_len, vocab_size]





# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import TensorDataset, DataLoader

# # ---------------------------
# # Meta-learner model definition
# # ---------------------------

# import torch
# import torch.nn as nn

# class MetaLearnerTokenWise(nn.Module):
#     def __init__(self,
#                  in_channels=3,      # number of base models
#                  num_classes=79,     # number of classes (treated as tokens)
#                  d_model=128,        # embedding dim per token
#                  nhead=4,
#                  num_layers=2,
#                  dropout=0.3):
#         super().__init__()

#         # 1x1 Conv1d to project model outputs per token to d_model
#         self.conv_in = nn.Conv1d(in_channels, d_model, kernel_size=1)

#         # Positional embeddings for each class position (token)
#         self.pos_embedding = nn.Parameter(torch.randn(1, num_classes, d_model))

#         # Transformer encoder layers
#         encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
#         self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

#         # Final linear layer per token to predict class logits
#         self.fc_out = nn.Linear(d_model, 1)  # Output 1 logit per token (per class)

#     def forward(self, x):
#         """
#         x: [B, in_channels=3, num_classes=79]
#         Output: [B, num_classes] logits (one per class)
#         """

#         # Input shape: [B, channels=3, length=79]
#         # Project channels -> d_model dim space
#         x = self.conv_in(x)  # [B, d_model, 79]

#         # Transformer expects [seq_len, batch, feature], so permute:
#         x = x.permute(2, 0, 1)  # [79, B, d_model]

#         # Add positional embeddings
#         pos_emb = self.pos_embedding.permute(1, 0, 2)  # [79, 1, d_model]
#         x = x + pos_emb  # broadcast along batch

#         # Pass through Transformer encoder
#         x = self.transformer_encoder(x)  # [79, B, d_model]

#         # Permute back: [B, 79, d_model]
#         x = x.permute(1, 0, 2)

#         # Predict a single logit per token (class)
#         logits = self.fc_out(x).squeeze(-1)  # [B, 79]

#         return logits



# # class MetaLearnerAttention(nn.Module):
# #     def __init__(self, feature_dim=79, num_classes=2):
# #         super(MetaLearnerAttention, self).__init__()
# #         self.attn = nn.MultiheadAttention(embed_dim=79, num_heads=1, batch_first=True)
# #         self.pool = nn.AdaptiveAvgPool1d(1)
# #         self.fc = nn.Linear(feature_dim, num_classes)

# #     def forward(self, x):  # x: [batch, 3, 79]
# #         attn_out, _ = self.attn(x, x, x)  # shape: [batch, 3, 79]
# #         pooled = self.pool(attn_out.transpose(1, 2))  # -> [batch, 79, 1] → avg over 3 → [batch, 79, 1]
# #         out = self.fc(pooled.squeeze(-1))  # [batch, 79] → [batch, num_classes]
# #         return out

# # class MetaLearnerAttention(nn.Module):
# #     def __init__(self, feature_dim=79, num_classes=2):
# #         super(MetaLearnerAttention, self).__init__()
# #         self.attn = nn.MultiheadAttention(embed_dim=feature_dim, num_heads=1, batch_first=True)
# #         self.fc = nn.Linear(feature_dim, num_classes)

# #     def forward(self, x):  # x: [batch, 3, 79]
# #         attn_out, _ = self.attn(x, x, x)       # [batch, 3, 79]
# #         x_avg = attn_out.mean(dim=1)           # [batch, 79]
# #         return self.fc(x_avg)                  # [batch, num_classes]

# # class MetaLearnerFC(nn.Module):
# #     def __init__(self, feature_dim=79, num_classes=2):
# #         super(MetaLearnerFC, self).__init__()
# #         # Define the fully connected layers
# #         self.fc1 = nn.Linear(feature_dim * 3, 3)  # 3 models, each with 79 features
# #         self.relu = nn.ReLU()
# #         self.fc2 = nn.Linear(3, num_classes)

# #     def forward(self, x):  # x: [batch, 3, 79]
# #         x = x.view(x.size(0), -1)  # Flatten to [batch, 3*79]
# #         x = self.fc1(x)            # [batch, 128]
# #         x = self.relu(x)           # [batch, 128]
# #         x = self.fc2(x)            # [batch, num_classes]
# #         return x

# # class WeightedSumModel(nn.Module):
# #     def __init__(self, feature_dim=79, num_vectors=3):
# #         super(WeightedSumModel, self).__init__()
# #         mean = 0
# #         std = 0  # You can adjust this spread as needed
# #         self.raw_weights = nn.Parameter(torch.randn(num_vectors))

# #     def forward(self, x):  # x: [batch, 3, 79]
# #         w = self.raw_weights
# #         w_min = w.min()
# #         w_max = w.max()

# #         norm_w = (w - w_min) / (w_max - w_min + 1e-8)
# #         norm_w = norm_w / (norm_w.sum() + 1e-8)
# #         norm_w = norm_w.view(1, -1, 1).to(x.device)  # Move to same device as x

# #         out = (x * norm_w).sum(dim=1)
# #         return out


-----------------**label encoder for Linear classifier**--------------------

In [8]:
from sklearn.preprocessing import LabelEncoder

train_annotations_df = pd.read_csv("/kaggle/input/fathomnet-dataset/annotations_Train (1).csv")
label_encoder = LabelEncoder().fit(train_annotations_df["label"].dropna())
train_annotations_df.head()

,path,label
0,/kaggle/input/fathomnet-dataset/DataSet/DataSe...,Sebastolobus
1,/kaggle/input/fathomnet-dataset/DataSet/DataSe...,Apostichopus leukothele
2,/kaggle/input/fathomnet-dataset/DataSet/DataSe...,Scotoplanes
3,/kaggle/input/fathomnet-dataset/DataSet/DataSe...,Keratoisis
4,/kaggle/input/fathomnet-dataset/DataSet/DataSe...,Keratoisis


In [9]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader

# Read labels directly from CSV
df = pd.read_csv("/kaggle/input/fathomnet-dataset/annotations_Train (1).csv")
labels = df["label"].tolist()
indices = list(range(len(df)))

# Stratified 40% split
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.9, random_state=12345)
split_idx, _ = next(splitter.split(indices, labels))

# Create datasets
dataset_lin = FathomNetDataset_indep(csv_path="/kaggle/input/fathomnet-dataset/annotations_Train (1).csv", transform=test_transforms, is_test=False)
dataset_other = FathomNetDataset_tkn(csv_path="/kaggle/input/fathomnet-dataset/annotations_Train (1).csv", transformation=test_transforms, is_test=False)

# Subset based on stratified split
subset_lin = Subset(dataset_lin, split_idx)
subset_other = Subset(dataset_other, split_idx)

# Dataloaders
test_loader_lin = DataLoader(subset_lin, batch_size=32, shuffle=False)
test_loader_other = DataLoader(subset_other, batch_size=32, shuffle=False)


In [10]:




import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader

# Create datasets
dataset_lin = FathomNetDataset_indep(csv_path="/kaggle/input/fathomnet-dataset/annotations_Test.csv", label_encoder = label_encoder, transform=test_transforms, is_test=True)
dataset_other = FathomNetDataset_tkn(csv_path="/kaggle/input/fathomnet-dataset/annotations_Test.csv", transformation=test_transforms, is_test=True)

# Dataloaders
test_loader_lin = DataLoader(dataset_lin, batch_size=32, shuffle=False)
test_loader_other = DataLoader(dataset_other, batch_size=32, shuffle=False)


In [11]:



# import pandas as pd
# from sklearn.model_selection import StratifiedShuffleSplit
# from torch.utils.data import Subset, DataLoader

# # Load CSV and labels for stratification
# csv_path = "/kaggle/input/fathomnet-dataset/annotations_Train (1).csv"
# full_data = pd.read_csv(csv_path)
# labels = full_data["label"].tolist()

# # Stratified split
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=2468)
# train_idx, val_idx = next(splitter.split(full_data["path"], labels))

# # Create full datasets
# dataset_lin_full = FathomNetDataset_indep(csv_path=csv_path, transform=test_transforms, is_test=False)
# dataset_other_full = FathomNetDataset_tkn(csv_path=csv_path, transformation=test_transforms, is_test=False)

# # Subset using stratified indices
# train_dataset_lin = Subset(dataset_lin_full, train_idx)
# val_dataset_lin = Subset(dataset_lin_full, val_idx)

# train_dataset_other = Subset(dataset_other_full, train_idx)
# val_dataset_other = Subset(dataset_other_full, val_idx)

# # DataLoaders
# train_loader_lin = DataLoader(train_dataset_lin, batch_size=32, shuffle=True)
# val_loader_lin = DataLoader(val_dataset_lin, batch_size=32, shuffle=False)

# train_loader_other = DataLoader(train_dataset_other, batch_size=32, shuffle=True)
# val_loader_other = DataLoader(val_dataset_other, batch_size=32, shuffle=False)


In [12]:
new_onehot = {int(onehot[vec].argmax()) : str(vec) for vec in onehot.keys()}

In [13]:
# import torch
# from tqdm import tqdm
# import torchvision.models as models  # ensure models refers to torchvision.models

# # --- Model loading ---
# # def load_models(weight_paths, **model_kwargs):
# #     """
# #     Instantiate and load three models from the given checkpoint paths.
# #     """
# #     model1 = FathomnetClassLSTM(**model_kwargs)
# #     model2 = FathomnetClassTree(**model_kwargs)
# #     # For linear, number of output classes should match your dataset (e.g., 79)
# #     model3 = FathomnetClass(num_classes=79)

# #     for m, w in zip((model1, model2, model3), weight_paths):
# #         ckpt = torch.load(w, map_location='cpu')
# #         m.load_state_dict(ckpt['model_state_dict'])
# #         m.eval()
# #     return model1, model2, model3

# def load_models(weight_paths, meta_weight_path='/kaggle/input/lstm_v1/pytorch/meta_classifier/1/Meta_Attention_1.pth', **model_kwargs):
#     """
#     Instantiate and load three base models and an optional meta-learner model.
#     """
#     model1 = FathomnetClassLSTM(**model_kwargs)
#     model2 = FathomnetClassTree(**model_kwargs)
#     model3 = FathomnetClass(num_classes=79)  # Linear model

#     for m, w in zip((model1, model2, model3), weight_paths):
#         ckpt = torch.load(w, map_location='cpu', )
#         m.load_state_dict(ckpt['model_state_dict'])
#         m.eval()

#     if meta_weight_path:
#         meta_model = MetaLearnerAttention(num_classes=79)
#         ckpt = torch.load(meta_weight_path, map_location='cpu')
#         meta_model.load_state_dict(ckpt['model_state_dict'])
#         meta_model.eval()
#         return [model1, model2, model3, meta_model]

#     return [model1, model2, model3]


# @torch.no_grad()
# # def ensemble_predict(ensemble_models, seq_images, lin_images,
# #                      onehot, label_encoder, labels_idx):
# #     """
# #     Predict by ensembling two sequence models and one linear model.
# #     """
# #     # Sequence-based logits
# #     logits1 = ensemble_models[0](seq_images)  # [B, T, V]
# #     logits2 = ensemble_models[1](seq_images)
# #     logits3 = ensemble_models[2](lin_images)  # [B, C]
# #     seq_logits = (4.2 * logits1 + 5.8 * logits2) / 10
# #     print(seq_logits.shape)
# #     seq_preds = seq_logits.argmax(dim=-1)    # [B, T]            

# #     # Decode sequence to single token per sample
# #     final_tokens = []
# #     for seq in seq_preds.tolist():
# #         cutoff = next((i for i, x in enumerate(seq) if x in (5, 6)), len(seq))
# #         prefix = seq[:cutoff]
# #         last_valid = next((x for x in reversed(prefix) if x in labels_idx), 0)
# #         final_tokens.append(last_valid)
# #     # Map tokens back to labels and one-hot encode for linear model
# #     final_labels = [label_encoder.transform([new_onehot[idx]])[0] for idx in final_tokens]
# #     # Linear model logits
# #     num_classes = logits3.size(1)
# #     onehot_dict = {i: torch.nn.functional.one_hot(torch.tensor(i % 79), num_classes).float() for i in range(151)}
# #     one_hot_seq = torch.stack([onehot_dict[i] for i in final_labels]).float().to(logits3.device)
# #     normalized_logits3 = (logits3 - logits3.min(dim=-1, keepdim=True).values) / (logits3.max(dim=-1, keepdim=True).values - logits3.min(dim=-1, keepdim=True).values)
# #     print(logits3.shape)
# #     # Average logits
# #     final_logits = normalized_logits3 + 0.5 * one_hot_seq
# #     predicted_classes = final_logits.argmax(dim=-1).tolist()
# #     return label_encoder.inverse_transform(predicted_classes).tolist()


# # def ensemble_predict(ensemble_models, seq_images, lin_images,
# #                      onehot, label_encoder, labels_idx):
# #     """
# #     Predict by ensembling two sequence models and one linear model.
# #     """
# #     # --- Sequence-based logits: [B, T, V] each
# #     logits1 = ensemble_models[0](seq_images)
# #     logits2 = ensemble_models[1](seq_images)
# #     seq_logits = (5.2 * logits1 + 4.8 * logits2) / 10  # [B, T, V]

# #     # --- For each class, take the max over time → [B, V]
# #     seq_max_logits = seq_logits.max(dim=1).values  # collapse T→classes

# #     # --- Reduce to only those classes in labels_idx → [B, len(labels_idx)=79]
# #     new_seq_logits = seq_max_logits[:, labels_idx]  # now matches linear-model output dims

# #     # --- Your original linear logits and normalization
# #     logits3 = ensemble_models[2](lin_images)        # [B, 79] after you train it
# #     normalized_logits3 = (logits3 - logits3.min(dim=-1, keepdim=True).values) / (logits3.max(dim=-1, keepdim=True).values - logits3.min(dim=-1, keepdim=True).values + 1e-8)
# #     new_seq_logits = (new_seq_logits - new_seq_logits.min(dim=-1, keepdim=True).values) / (new_seq_logits.max(dim=-1, keepdim=True).values - new_seq_logits.min(dim=-1, keepdim=True).values + 1e-8)

# #     # --- Combine
# #     print(normalized_logits3)
# #     print(new_seq_logits)
# #     final_logits = 0.27 * normalized_logits3 + 0.73 * new_seq_logits.to(logits3.device)
# #     preds_idx    = final_logits.argmax(dim=-1).tolist()
# #     return label_encoder.inverse_transform(preds_idx).tolist()


# def ensemble_predict(ensemble_models, seq_images, lin_images,
#                      onehot, label_encoder, labels_idx):
#     """
#     Predict using 2 sequence models, 1 linear model, and a meta-learner.
#     """
#     # --- Sequence-based logits: [B, T, V]
#     logits1 = ensemble_models[0](seq_images)
#     logits2 = ensemble_models[1](seq_images)
#     seq_logits = (5.2 * logits1 + 4.8 * logits2) / 10  # [B, T, V]

#     logits1_argmax = logits1.argmax(dim=-1)
#     logits2_argmax = logits2.argmax(dim=-1)

#     final_tokens1 = []
#     final_tokens2 = []
#     for seq in logits1_argmax.tolist():
#         cutoff = next((i for i, x in enumerate(seq) if x in (5, 6)), len(seq))
#         prefix = seq[:cutoff]
#         last_valid = next((x for x in reversed(prefix) if x in labels_idx), 0)
#         final_tokens1.append(onehot[new_onehot[last_valid]])
#     for seq in logits2_argmax.tolist():
#         cutoff = next((i for i, x in enumerate(seq) if x in (5, 6)), len(seq))
#         prefix = seq[:cutoff]
#         last_valid = next((x for x in reversed(prefix) if x in labels_idx), 0)
#         final_tokens2.append(onehotpnew_onehot[last_valid]])

#     final_tokens1 = torch.tensor(final_tokens1)
#     print(final_tokens1)
#     final_tokens2 = torch.tensor(final_tokens2)
#     logits1 = final_tokens1[:, :, labels_idx]
#     logits2 = final_tokens2[:, :, labels_idx]

#     # --- Collapse time dim by max over T: [B, V]
#     seq_max_logits = seq_logits.max(dim=1).values

#     # --- Reduce to [B, 79] using only selected indices
#     seq_selected = seq_max_logits[:, labels_idx]

#     # --- Linear model logits
#     logits3 = ensemble_models[2](lin_images)

#     # Normalize both to [0, 1]
#     def normalize(x):
#         return (x - x.min(dim=-1, keepdim=True).values) / (x.max(dim=-1, keepdim=True).values - x.min(dim=-1, keepdim=True).values + 1e-8)

#     # seq_selected = normalize(seq_selected)
#     # logits3 = normalize(logits3)

#     # --- Add dummy third vector (or replace if you have a 3rd)
#     dummy_tree_logits = torch.zeros_like(logits3)  # Replace with actual 3rd model if needed

#     # --- Stack for meta-learner
#     x_meta = torch.stack([logits3, logits1, logits2], dim=1)  # [B, 3, 79]

#     # --- Predict
#     final_logits = ensemble_models[3](x_meta.to(logits3.device))
#     preds_idx = final_logits.argmax(dim=-1).tolist()
#     return label_encoder.inverse_transform(preds_idx).tolist()



# # --- Setup ---
# # Determine special token indices
# stop_tkn_idx = torch.argmax(torch.tensor(
#     onehot.get('spl_stp', torch.zeros_like(torch.tensor(onehot['-']))))
# ).item()
# pad_tkn_idx = torch.argmax(torch.tensor(
#     onehot.get('-', torch.zeros_like(torch.tensor(onehot['-']))))
# ).item()

# model_kwargs = {
#     'input_dim': 1280,
#     'hidden_size': 512,
#     'vocab_size': 151,
#     'stop_tkn_idx': stop_tkn_idx,
#     'pad_tkn_idx': pad_tkn_idx,
#     'max_len': 7,
# }
# # Paths to saved checkpoints
# weight_paths = ['/kaggle/input/lstm_v1/pytorch/v1/5/LSTM_3k_4.pth', 
#                 '/kaggle/input/lstm_v1/pytorch/n-ary_treelstm/5/TreeLSTM_Nary_512_7ok.pth',
#                 '/kaggle/input/lstm_v1/pytorch/linear/1/Linear_3.pth']

# # Load and move to device
# ensemble_models = load_models(weight_paths, **model_kwargs)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# for m in ensemble_models:
#     m.to(device)

# # --- Inference using separate loaders ---
# predictions, ids = [], []
# for (imgs_seq, paths_seq), (imgs_lin, paths_lin) in tqdm(
#         zip(test_loader_other, test_loader_lin),
#         desc="Processing batches"
# ):
#     # Ensure both loaders yield the same ordering
#     assert paths_seq == paths_lin, \
#         "Mismatched batch ordering between sequence and linear loaders"

#     imgs_seq = imgs_seq.to(device)
#     imgs_lin = imgs_lin.to(device)

#     preds = ensemble_predict(
#         ensemble_models,
#         imgs_seq,
#         imgs_lin,
#         onehot,
#         label_encoder,
#         labels_idx
#     )
#     predictions.extend(preds)
#     # Extract numeric ID from filename pattern
#     ids.extend([
#         int(p.split('/')[-1].split('_')[1].split('.')[0])
#         for p in paths_seq
#     ])

# print(f"Total samples: {len(ids)}")
# print(f"First ID: {ids[0]}, First prediction: {predictions[0]}")

In [14]:




# import torch
# from tqdm import tqdm
# import torchvision.models as models  # ensure models refers to torchvision.models
# import pandas as pd
# import torch.nn.functional as F


# # --- Model loading ---
# def load_models(weight_paths,
#                 meta_weight_path='/kaggle/input/lstm_v1/pytorch/meta_classifier/10/WeightedSumModel_new_2.pth',
#                 **model_kwargs):
#     """
#     Instantiate and load three base models and an optional meta-learner model.
#     """
#     model1 = FathomnetClassLSTM(**model_kwargs)
#     model2 = FathomnetClassTree(**model_kwargs)
#     model3 = FathomnetClass(num_classes=79)

#     for m, w in zip((model1, model2, model3), weight_paths):
#         ckpt = torch.load(w, map_location='cpu')
#         m.load_state_dict(ckpt['model_state_dict'])
#         m.eval()

#     if meta_weight_path:
#         meta_model = WeightedSumModel()
#         ckpt = torch.load(meta_weight_path, map_location='cpu')
#         meta_model.load_state_dict(ckpt['model_state_dict'])
#         meta_model.eval()
#         return [model1, model2, model3, meta_model]

#     return [model1, model2, model3]

# # --- Prediction function ---
# @torch.no_grad()
# def ensemble_predict(ensemble_models, seq_images, lin_images,
#                      onehot, label_encoder, labels_idx):
#     """
#     Predict using 2 sequence models, 1 linear model, and a meta-learner.
#     """
#     logits1 = ensemble_models[0](seq_images)    # LSTM
#     logits2 = ensemble_models[1](seq_images)    # Tree

#     arg1 = logits1.argmax(dim=-1).tolist()
#     arg2 = logits2.argmax(dim=-1).tolist()

#     final1, final2 = [], []
#     for seq in arg1:
#         cutoff = next((i for i, x in enumerate(seq) if x in (5, 6)), len(seq))
#         prefix = seq[:cutoff]
#         last = next((x for x in reversed(prefix) if x in labels_idx), 0)
#         final1.append(onehot[new_onehot[last]])
#     for seq in arg2:
#         cutoff = next((i for i, x in enumerate(seq) if x in (5, 6)), len(seq))
#         prefix = seq[:cutoff]
#         last = next((x for x in reversed(prefix) if x in labels_idx), 0)
#         final2.append(onehot[new_onehot[last]])

#     logits3 = ensemble_models[2](lin_images)
#     logits3 = F.normalize(logits3, dim=-1)
    
#     tensor_final1 = torch.tensor(final1, device=logits3.device, dtype=logits3.dtype)
#     tensor_final1 = tensor_final1[:, labels_idx]
#     tensor_final1 = F.normalize(tensor_final1, dim=-1)
    
#     tensor_final2 = torch.tensor(final2, device=logits3.device, dtype=logits3.dtype)
#     tensor_final2 = tensor_final2[:, labels_idx]
#     tensor_final2 = F.normalize(tensor_final2, dim=-1)

#     stacked = torch.stack([logits3, tensor_final1, tensor_final2], dim=1)  # [batch, 3, dim]
#     final_logits = ensemble_models[3](stacked)  # WeightedSumModel
    
#     # final_logits = (3.2 * tensor_final1 + 3.5 * tensor_final2 + 2.7 * logits3) / 9.6
#     preds_idx = final_logits.argmax(dim=-1).tolist()
#     return label_encoder.inverse_transform(preds_idx).tolist()

# # --- Setup ---
# stop_tkn_idx = torch.argmax(torch.tensor(
#     onehot.get('spl_stp', torch.zeros_like(torch.tensor(onehot['-'])))
# )).item()
# pad_tkn_idx = torch.argmax(torch.tensor(
#     onehot.get('-', torch.zeros_like(torch.tensor(onehot['-'])))
# )).item()

# model_kwargs = {
#     'input_dim': 1280,
#     'hidden_size': 512,
#     'vocab_size': 151,
#     'stop_tkn_idx': stop_tkn_idx,
#     'pad_tkn_idx': pad_tkn_idx,
#     'max_len': 7,
# }

# weight_paths = [
#     '/kaggle/input/lstm_v1/pytorch/v1/6/LSTM_ens_3p.pth',
#     '/kaggle/input/lstm_v1/pytorch/n-ary_treelstm/6/TreeLSTM_Nary_ens_3p.pth',
#     '/kaggle/input/lstm_v1/pytorch/linear/4/Linear_ens_3.pth'
# ]

# ensemble_models = load_models(weight_paths, **model_kwargs)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# for m in ensemble_models:
#     m.to(device)

# # --- Inference toggle ---
# include_paths_in_output = False  # Set to False to exclude file path column

# # --- Inference ---
# predictions, ids = [], []
# paths_out = [] if include_paths_in_output else None

# for (imgs_seq, paths_seq), (imgs_lin, paths_lin) in tqdm(
#         zip(test_loader_other, test_loader_lin),
#         desc="Processing batches"
# ):
#     assert paths_seq == paths_lin, "Batch ordering mismatch"
#     imgs_seq, imgs_lin = imgs_seq.to(device), imgs_lin.to(device)

#     preds = ensemble_predict(
#         ensemble_models,
#         imgs_seq,
#         imgs_lin,
#         onehot,
#         label_encoder,
#         labels_idx
#     )
#     predictions.extend(preds)
#     # ids.extend(paths_seq)
#     ids.extend(int(p.split('_')[-1].split('.')[0]) for p in paths_seq)
    
#     if include_paths_in_output:
#         paths_out.extend(paths_seq)

# print(f"Total samples: {len(ids)}")
# print(f"First ID: {ids[0]}, First prediction: {predictions[0]}")

# # --- Create submission DataFrame ---
# data = {'id': ids, 'prediction': predictions}
# if include_paths_in_output:
#     data['path'] = paths_out

# submission_df = pd.DataFrame(data)
# print(submission_df.head())


In [15]:





import torch
from tqdm import tqdm
import torchvision.models as models
import pandas as pd
import torch.nn.functional as F
import timm

# --- Model loading ---
def load_models(weight_paths,
                meta_weight_path=None,
                device='cpu',
                **model_kwargs):
    model1 = FathomnetClassLSTM(**model_kwargs)
    model2 = FathomnetClassTree(**model_kwargs)
    model3 = FathomnetClass(num_classes=79)
    model4 = FathomnetClassResnet(input_dim=2048, hidden_size=512, vocab_size=151,
                                 stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
                                 pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(), max_len=7)
    model5 = FathomnetClassSwin(input_dim=1536, hidden_size=512, vocab_size=151,
                                 stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
                                 pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(), max_len=7)

    for m, w in zip((model1, model2, model3, model4), weight_paths):
        ckpt = torch.load(w, map_location='cpu')
        m.load_state_dict(ckpt['model_state_dict'])
        m.eval()
        m.to(device)  # Move model to device here AFTER loading and eval

    if meta_weight_path:
        meta_model = MetaLearnerTokenWise()
        ckpt = torch.load(meta_weight_path, map_location='cpu')
        meta_model.load_state_dict(ckpt['model_state_dict'])
        meta_model.eval()
        meta_model.to(device)  # Move meta_model to device
        return [model1, model2, model3, model4, model5, meta_model]

    return [model1, model2, model3, model4, model5]

@torch.no_grad()
def ensemble_predict(ensemble_models, seq_images, lin_images,
                     onehot, label_encoder, labels_idx, w = [0.35, 0.25, 0.16, 0.25, 0.13]):
    onehot_argmax = {v.values.argmax(): k for k, v in onehot.items()}
    new_argmax = {new_idx: onehot_argmax[orig_idx] for new_idx, orig_idx in enumerate(labels_idx)}
    
    def predict_single(seq_img, lin_img):
        logits1 = ensemble_models[0](seq_img)  # [1, 7, 151]
        logits2 = ensemble_models[1](seq_img)  # [1, 7, 151]
        logits3 = ensemble_models[2](seq_img)
        logits4 = ensemble_models[3](seq_img)  # [1, 7, 151]
        logits5 = ensemble_models[4](seq_img)  # [1, 7, 151]

        def get_k(logits):
            pc = logits[0].argmax(dim=1).tolist()
            cutoff = next((i+1 for i in reversed(range(len(pc))) if pc[i] not in (5,6)), len(pc))
            prefix = pc[:cutoff]
            return next((pos for pos in reversed(range(cutoff)) if prefix[pos] in labels_idx), 0)

        k1, k2, k4, k5 = get_k(logits1), get_k(logits2), get_k(logits4), get_k(logits5)

        # f1 = logits1[0, k1, labels_idx]
        # f2 = logits2[0, k2, labels_idx]
        # f3 = logits3
        # f4 = logits4[0, k4, labels_idx]
        # f5 = logits5[0, k5, labels_idx]

        f1 = (logits1[0, k1, labels_idx] - logits1[0, k1, labels_idx].mean()) / (logits1[0, k1, labels_idx].std() + 1e-6)
        f2 = (logits2[0, k2, labels_idx] - logits2[0, k2, labels_idx].mean()) / (logits2[0, k2, labels_idx].std() + 1e-6)
        f3 = (logits3 - logits3.mean()) / (logits3.std() + 1e-6)
        f4 = (logits4[0, k4, labels_idx] - logits4[0, k4, labels_idx].mean()) / (logits4[0, k4, labels_idx].std() + 1e-6)
        f5 = (logits5[0, k5, labels_idx] - logits5[0, k5, labels_idx].mean()) / (logits5[0, k5, labels_idx].std() + 1e-6)

        # f_logits = torch.stack([f2, f4, f1], dim=0).unsqueeze(0)  # [1, 3, 79]
        # final_logits = ensemble_models[4](f_logits)  # [1, 79]

        final_logits = w[0] * f1 + w[1] * f2 + w[2] * f3 + w[3] * f4 + w[4] * f5

        idx = final_logits.argmax(dim=-1).item()

        return new_argmax.get(idx, None)

    batch_size = seq_images.size(0)
    preds = []
    for i in range(batch_size):
        si = seq_images[i:i+1]
        li = lin_images[i:i+1]
        preds.append(predict_single(si, li))
    return preds


# --- Setup ---
stop_tkn_idx = torch.argmax(torch.tensor(
    onehot.get('spl_stp', torch.zeros_like(torch.tensor(onehot['-'])))
)).item()
pad_tkn_idx = torch.argmax(torch.tensor(
    onehot.get('-', torch.zeros_like(torch.tensor(onehot['-'])))
)).item()

model_kwargs = {
    'input_dim': 1280,
    'hidden_size': 512,
    'vocab_size': 151,
    'stop_tkn_idx': stop_tkn_idx,
    'pad_tkn_idx': pad_tkn_idx,
    'max_len': 7,
}
# weight_paths = [
#     '/kaggle/input/retrained-models/effi_lstm/effi_lstm_ens_8o',
#     '/kaggle/input/retrained-models/effi_tree/effi_treelstm_ens_5o',
#     '/kaggle/input/retrained-models/swin/swin_trans_treelstm_ens_4o_complete.pth',
#     '/kaggle/input/retrained-models/resnest/resnest_treelstm_ens_5o'
# ]

weight_paths = [
    '/kaggle/input/lstm_v1/pytorch/v1/6/LSTM_3k_4.pth',
    '/kaggle/input/lstm_v1/pytorch/n-ary_treelstm/6/TreeLSTM_Nary_512_7ok.pth',
    '/kaggle/input/lstm_v1/pytorch/linear/4/Linear_3.pth',
    '/kaggle/input/retrained-models/resnest/resnest_treelstm_ens_5o',
    '/kaggle/input/retrained-models/swin/swin_trans_treelstm_ens_4o_complete.pth'
]

ensemble_models = load_models(weight_paths, **model_kwargs)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for m in ensemble_models:
    m.to(device)

# --- Inference toggle ---
include_paths_in_output = False

# --- Inference ---
predictions = []
ids = []
paths_out = [] if include_paths_in_output else None

for (imgs_seq, paths_seq), (imgs_lin, paths_lin) in tqdm(
        zip(test_loader_other, test_loader_lin),
        desc="Processing batches"
):
    assert paths_seq == paths_lin, "Batch ordering mismatch"
    imgs_seq, imgs_lin = imgs_seq.to(device), imgs_lin.to(device)

    batch_preds = ensemble_predict(
        ensemble_models,
        imgs_seq,
        imgs_lin,
        onehot,
        label_encoder,
        labels_idx
    )
    predictions.extend(batch_preds)
    ids.extend(int(p.split('_')[-1].split('.')[0]) for p in paths_seq)

    if include_paths_in_output:
        paths_out.extend(paths_seq)

print(f"Total samples: {len(ids)}")
print(f"First ID: {ids[0]}, First prediction: {predictions[0]}")

# --- Create submission DataFrame ---
data = {
    'id': ids,
    'prediction': predictions
}
if include_paths_in_output:
    data['path'] = paths_out

# Sanity check and truncate any mismatched lengths
lengths = {k: len(v) for k, v in data.items()}
min_len = min(lengths.values())
for k in data:
    data[k] = data[k][:min_len]

submission_df = pd.DataFrame(data)
print(submission_df.head())


Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 192MB/s]


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

/tmp/ipykernel_19/963655924.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(w, map_location='cpu')
Processing batches: 25it [02:53,  6.94s/it]

Total samples: 788
First ID: 1, First prediction: Pandalus amplus
   id       prediction
0   1  Pandalus amplus
1   2       Funiculina
2   3       Funiculina
3   4       Funiculina
4   5       Funiculina


In [16]:
# import torch
# from tqdm import tqdm
# import pandas as pd
# from collections import Counter
# # Assuming FathomnetClassLSTM, FathomnetClassTree, FathomnetClass, FathomnetClassResnet,
# # FathomnetClassSwin and onehot, labels_idx, test_loader_other, test_loader_lin are defined elsewhere

# # --- Model loading ---
# def load_models(weight_paths, device='cpu', **model_kwargs):
#     model1 = FathomnetClassLSTM(**model_kwargs)
#     model2 = FathomnetClassTree(**model_kwargs)
#     model3 = FathomnetClass(num_classes=79)
#     model4 = FathomnetClassResnet(
#         input_dim=2048,
#         hidden_size=model_kwargs.get('hidden_size', 512),
#         vocab_size=model_kwargs['vocab_size'],
#         stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
#         pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(),
#         max_len=model_kwargs['max_len']
#     )
#     model5 = FathomnetClassSwin(
#         input_dim=1536,
#         hidden_size=model_kwargs.get('hidden_size', 512),
#         vocab_size=model_kwargs['vocab_size'],
#         stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
#         pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(),
#         max_len=model_kwargs['max_len']
#     )

#     for m, w in zip((model1, model2, model3, model4, model5), weight_paths):
#         ckpt = torch.load(w, map_location='cpu')
#         m.load_state_dict(ckpt['model_state_dict'])
#         m.eval()
#         m.to(device)

#     return [model1, model2, model3, model4, model5]

# @torch.no_grad()
# def ensemble_predict(ensemble_models, seq_images, lin_images, onehot, labels_idx):
#     onehot_argmax = {v.values.argmax(): k for k, v in onehot.items()}
#     new_argmax = {new_idx: onehot_argmax[orig_idx] for new_idx, orig_idx in enumerate(labels_idx)}

#     def get_pred(logits, use_seq=True):
#         if use_seq:
#             pc = logits[0].argmax(dim=1).tolist()
#             cutoff = next((i+1 for i in reversed(range(len(pc))) if pc[i] not in (5, 6)), len(pc))
#             prefix = pc[:cutoff]
#             k = next((pos for pos in reversed(range(cutoff)) if prefix[pos] in labels_idx), 0)
#             vals = logits[0, k, labels_idx]
#         else:
#             vals = logits[0]
#         idx = vals.argmax().item()
#         return new_argmax.get(idx, None)

#     preds = []
#     for si, li in zip(seq_images, lin_images):
#         si, li = si.unsqueeze(0), li.unsqueeze(0)
#         # get logits from each model
#         logits1 = ensemble_models[0](si)
#         logits2 = ensemble_models[1](si)
#         logits3 = ensemble_models[2](li)
#         logits4 = ensemble_models[3](si)
#         logits5 = ensemble_models[4](si)
        
#         # individual model predictions
#         p1 = get_pred(logits1, use_seq=True)
#         p2 = get_pred(logits2, use_seq=True)
#         p3 = get_pred(logits3, use_seq=False)
#         p4 = get_pred(logits4, use_seq=True)
#         p5 = get_pred(logits5, use_seq=True)
        
#         # majority voting
#         votes = [p1, p2, p3, p4, p5]
#         weights = [5.5, 4.5, 1, 4, 1.5]
#         vote_scores = {}
#         for pred, weight in zip(votes, weights):
#             vote_scores[pred] = vote_scores.get(pred, 0) + weight
#         final_pred = max(vote_scores.items(), key=lambda x: x[1])[0]
#         preds.append(final_pred)
#     return preds

# # --- Setup ---
# stop_tkn_idx = torch.argmax(torch.tensor(onehot['spl_stp'])).item()
# pad_tkn_idx = torch.argmax(torch.tensor(onehot['-'])).item()
# model_kwargs = {'input_dim': 1280, 'hidden_size': 512, 'vocab_size': 151, 'stop_tkn_idx': stop_tkn_idx, 'pad_tkn_idx': pad_tkn_idx, 'max_len': 7}
# weight_paths = [
#     '/kaggle/input/retrained-models/effi_lstm/effi_lstm_ens_8o',
#     '/kaggle/input/retrained-models/effi_tree/effi_treelstm_ens_5o',
#     '/kaggle/input/retrained-models/effi_lin_64/effi_ens_2o_complete',
#     '/kaggle/input/retrained-models/resnest/resnest_treelstm_ens_5o',
#     '/kaggle/input/retrained-models/swin/swin_trans_treelstm_ens_4o_complete.pth'
# ]

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ensemble_models = load_models(weight_paths, device=device, **model_kwargs)

# # --- Inference ---
# predictions, ids = [], []
# for (imgs_seq, paths_seq), (imgs_lin, paths_lin) in tqdm(zip(test_loader_other, test_loader_lin), desc="Processing batches"):
#     assert paths_seq == paths_lin, "Batch ordering mismatch"
#     imgs_seq, imgs_lin = imgs_seq.to(device), imgs_lin.to(device)
#     batch_preds = ensemble_predict(ensemble_models, imgs_seq, imgs_lin, onehot, labels_idx)
#     predictions.extend(batch_preds)
#     ids.extend([int(p.split('_')[-1].split('.')[0]) for p in paths_seq])

# # prepare submission
# submission_df = pd.DataFrame({'id': ids, 'prediction': predictions})
# print(submission_df.head())


In [17]:




# import torch
# from tqdm import tqdm
# import pandas as pd
# import torch.nn.functional as F

# # --- 1. Load your ensemble models as before ---
# def load_models(weight_paths,
#                 meta_weight_path=None,
#                 device='cpu',
#                 **model_kwargs):
#     model1 = FathomnetClassLSTM(**model_kwargs)
#     model2 = FathomnetClassTree(**model_kwargs)
#     model3 = FathomnetClassSwin(input_dim=1536, hidden_size=512, vocab_size=151,
#                                    stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
#                                    pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(), max_len=7)
#     model4 = FathomnetClassResnet(input_dim=2048, hidden_size=512, vocab_size=151,
#                                  stop_tkn_idx=torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
#                                  pad_tkn_idx=torch.argmax(torch.tensor(onehot['-'])).item(), max_len=7)

#     for m, w in zip((model1, model2, model3, model4), weight_paths):
#         ckpt = torch.load(w, map_location='cpu')
#         m.load_state_dict(ckpt['model_state_dict'])
#         m.eval()
#         m.to(device)  # Move model to device here AFTER loading and eval

#     if meta_weight_path:
#         meta_model = MetaLearnerTokenWise()
#         ckpt = torch.load(meta_weight_path, map_location='cpu')
#         meta_model.load_state_dict(ckpt['model_state_dict'])
#         meta_model.eval()
#         meta_model.to(device)  # Move meta_model to device
#         return [model1, model2, model3, model4, meta_model]

#     return [model1, model2, model3, model4]

# @torch.no_grad()
# def ensemble_predict_single(models, x, w, labels_idx):
#     # Compute the three model logits and normalize per your logic:
#     logits1 = models[0](x)  # [1,T,V]
#     logits2 = models[1](x)
#     logits3 = models[2](x)
#     logits4 = models[3](x)

#     def extract(logits):
#         pc = logits[0].argmax(dim=1).tolist()
#         cutoff = next((i+1 for i in reversed(range(len(pc))) if pc[i] not in (5,6)), len(pc))
#         pos = next((i for i in reversed(range(cutoff)) if pc[i] in labels_idx), 0)
#         vec = logits[0, pos, labels_idx]
#         return (vec - vec.mean()) / (vec.std() + 1e-6)

#     f1 = extract(logits1)
#     f2 = extract(logits2)
#     f3 = extract(logits3)
#     f4 = extract(logits4)

#     return f1, f3, f2, f4

# # --- 2. Setup & load models ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model_kwargs = { 'input_dim':1280, 'hidden_size':512, 'vocab_size':151,
#                  'stop_tkn_idx':torch.argmax(torch.tensor(onehot['spl_stp'])).item(),
#                  'pad_tkn_idx':torch.argmax(torch.tensor(onehot['-'])).item(),
#                  'max_len':7 }

# weight_paths = [
#     '/kaggle/input/retrained-models/effi_lstm/effi_lstm_ens_8o',
#     '/kaggle/input/retrained-models/effi_tree/effi_treelstm_ens_5o',
#     '/kaggle/input/retrained-models/swin/swin_trans_treelstm_ens_4o_complete.pth',
#     '/kaggle/input/retrained-models/resnest/resnest_treelstm_ens_5o'
# ]
# ensemble_models = load_models(weight_paths, **model_kwargs)
# for m in ensemble_models:
#     m.to(device)

# # --- 3. Collect all normalized logits and true labels once ---
# all_f1, all_f2, all_f3, all_f4 = [], [], [], []
# true_labels_list = []

# for (seq_imgs, seq_lbls), (lin_imgs, lin_lbls) in tqdm(
#         zip(val_loader_other, val_loader_lin),
#         total=len(val_loader_other),
#         desc="Collecting logits",
#         dynamic_ncols=True
# ):
#     assert seq_lbls == lin_lbls, "Mismatch in labels"
#     seq_imgs = seq_imgs.to(device)

#     for i in range(seq_imgs.size(0)):
#         x = seq_imgs[i:i+1]
#         f1, f3, f2, f4 = ensemble_predict_single(
#             ensemble_models, x, None, labels_idx
#         )
#         all_f1.append(f1)           # LSTM (normalized)
#         all_f2.append(f2) # Linear (raw logits)
#         all_f3.append(f3)           # TreeLSTM (normalized)
#         all_f4.append(f4)           # ResNet (normalized)
#         true_labels_list.append(lin_lbls[i])  # raw string label

# # Stack into tensors of shape [N, num_classes]
# all_f1 = torch.stack(all_f1)  # LSTM
# all_f2 = torch.stack(all_f2)  # Linear
# all_f3 = torch.stack(all_f3)  # TreeLSTM
# all_f4 = torch.stack(all_f4)  # ResNet



In [18]:
# import torch

# save_path = 'logits.pth'

# torch.save({
#     'all_f1': all_f1,                  # LSTM features
#     'all_f2': all_f2,                  # Linear logits
#     'all_f3': all_f3,                  # TreeLSTM features
#     'all_f4': all_f4,                  # ResNet features
#     'true_labels_list': true_labels_list  # Raw string labels
# }, save_path)

# print(f"✅ Saved to {save_path}")


In [19]:
# logits = torch.load('/kaggle/input/logits-50-2468/logits (1).pth')
# all_f1 = logits['all_f1']
# all_f2 = logits['all_f2']
# all_f3 = logits['all_f3']
# all_f4 = logits['all_f4']
# true_labels_list = logits['true_labels_list']

In [20]:
# import torch

# torch.cuda.empty_cache()             # Releases unused memory back to the GPU
# torch.cuda.ipc_collect()            # Cleans up inter-process memory (if any)
# torch.cuda.reset_peak_memory_stats()
# torch.cuda.reset_accumulated_memory_stats()


In [21]:
# data = torch.load('/kaggle/working/logits.pth')
# all_f1 = data['all_f1']
# all_f2 = data['all_f2']
# all_f3 = data['all_f3']
# all_f4 = data['all_f4']
# true_labels_list = data['true_labels_list']

In [22]:
# import torch
# from tqdm import tqdm
# import pandas as pd
# import matplotlib.pyplot as plt

# # --- device & batch setup ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# use_gpu = (device.type == 'cuda')
# batch_size = 1024

# # --- move tensors to device ---
# if use_gpu:
#     all_f1 = all_f1.to(device)
#     all_f2 = all_f2.to(device)
#     all_f3 = all_f3.to(device)
#     all_f4 = all_f4.to(device)
# else:
#     all_f1 = all_f1.cpu()
#     all_f2 = all_f2.cpu()
#     all_f3 = all_f3.cpu()
#     all_f4 = all_f4.cpu()

# # --- prepare mapping once ---
# onehot_argmax = {v.values.argmax(): k for k, v in onehot.items()}
# inv_map = {new_idx: onehot_argmax[orig_idx] for new_idx, orig_idx in enumerate(labels_idx)}

# # --- validate label count ---
# assert len(true_labels_list) == all_f1.size(0), "Label/logit count mismatch"

# # --- 4. Generate weight combos ---
# step = 0.01
# min_w, max_w = 0, 1
# weight_combos = []
# w1 = min_w
# while w1 <= max_w:
#     w2 = min_w
#     while w2 <= max_w:
#         w3 = min_w
#         while w3 <= max_w:
#             w4 = round(1.0 - w1 - w2 - w3, 4)
#             if 0 <= w4 <= 1.0:
#                 weight_combos.append((round(w1,4), round(w2,4), round(w3,4), round(w4,4)))
#             w3 = round(w3 + step, 4)
#         w2 = round(w2 + step, 4)
#     w1 = round(w1 + step, 4)

# # --- 5. Search best weights ---
# results = []
# best_acc = 0.0
# best_w = None
# N = all_f1.size(0)

# for w1, w2, w3, w4 in tqdm(weight_combos, desc="Searching weights", dynamic_ncols=True):
#     correct = 0
#     for start in range(0, N, batch_size) if use_gpu else [0]:
#         end = min(start + batch_size, N)
#         # f1 = all_f1[start:end]
#         # f2 = all_f2[start:end]
#         # f3 = all_f3[start:end].squeeze()
#         # f4 = all_f4[start:end]

#         f1 = (all_f1[start:end] - all_f1[start:end].mean()) / (all_f1[start:end].std() + 1e-6)
#         f2 = (all_f2[start:end] - all_f2[start:end].mean()) / (all_f2[start:end].std() + 1e-6)
#         f3 = (all_f3[start:end] - all_f3[start:end].mean()) / (all_f3[start:end].std() + 1e-6)
#         f4 = (all_f4[start:end] - all_f4[start:end].mean()) / (all_f4[start:end].std() + 1e-6)
        
#         final_logits = w1 * f1 + w2 * f2 + w3 * f3 + w4 * f4
#         preds = final_logits.argmax(dim=-1).tolist()

#         true_batch = true_labels_list[start:end]
#         pred_labels = [inv_map[p] for p in preds]

#         correct += sum(p == t for p, t in zip(pred_labels, true_batch))
#         if not use_gpu:
#             break

#     acc = correct / N
#     results.append((w1, w2, w3, w4, acc))
#     if acc > best_acc:
#         best_acc, best_w = acc, (w1, w2, w3, w4)

# # New: Print top 5 best weights and accuracies
# top5 = sorted(results, key=lambda x: x[-1], reverse=True)[:5]

# print("\nTop 5 best weights and accuracies:")
# for i, (w1, w2, w3, w4, acc) in enumerate(top5, 1):
#     print(f"{i}: Weights (LSTM={w1:.2f}, Linear={w2:.2f}, Tree={w3:.2f}, ResNet={w4:.2f}) - Accuracy: {acc:.4f}")

# # --- 6. Display results ---
# df_results = pd.DataFrame(results, columns=['LSTM','Linear','Tree','ResNet','accuracy'])
# print(df_results)
# print(f"\n✅ Best Weights {best_w}, Accuracy {best_acc:.4f}")

# # --- 7. 3D Scatter Plot ---
# fig = plt.figure(figsize=(10,7))
# ax = fig.add_subplot(111, projection='3d')
# sc = ax.scatter(df_results['Linear'], df_results['Tree'], df_results['ResNet'],
#                 c=df_results['accuracy'], cmap='viridis', s=40)
# ax.set_xlabel('Linear')
# ax.set_ylabel('Tree')
# ax.set_zlabel('ResNet')
# ax.set_title('Accuracy vs Linear/Tree/ResNet Weights')
# fig.colorbar(sc, ax=ax, label='Accuracy')
# plt.show()

# # --- 8. 2D Heatmap ---
# fixed_lstm = best_w[0]
# df_heat = df_results[df_results['LSTM']==fixed_lstm]
# pivot = df_heat.pivot(index='Tree', columns='Linear', values='accuracy')
# plt.figure(figsize=(10,8))
# plt.imshow(pivot, origin='lower', aspect='auto', cmap='viridis')
# plt.xlabel('Linear Weight')
# plt.ylabel('Tree Weight')
# plt.title(f'Heatmap (LSTM={fixed_lstm:.2f})')
# plt.colorbar(label='Accuracy')
# plt.xticks(range(len(pivot.columns)), [f"{x:.2f}" for x in pivot.columns], rotation=90)
# plt.yticks(range(len(pivot.index)), [f"{y:.2f}" for y in pivot.index])
# plt.tight_layout()
# plt.show()


In [23]:
# inv_map = {v.values.argmax(): k for k, v in onehot.items()}
# len(inv_map)

In [24]:
# import torch
# import pandas as pd
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from tqdm import tqdm

# # --- Hyperparameters ---
# step = 0.01
# min_w, max_w = -3, 3  # weights range from 0 to 1

# # --- 1. Generate weight combinations ---
# weight_combos = []

# w1 = min_w
# while w1 <= max_w:
#     w2 = min_w
#     while w2 <= max_w:
#         w3 = 1.0 - w1 - w2
#         # Only keep combos where w3 >= 0 (weights sum to exactly 1)
#         if w3 >= 0:
#             weight_combos.append((round(w1, 4), round(w2, 4), round(w3, 4)))
#         w2 = round(w2 + step, 4)
#     w1 = round(w1 + step, 4)

# # --- 2. Search best weights ---
# results = []
# best_acc = 0.0
# best_w = None

# for w1, w2, w3 in tqdm(weight_combos, desc="Searching weights", dynamic_ncols=True):
#     final_logits = w1 * all_f1 + w2 * all_f2 + w3 * all_f4  # [N, V]
#     preds = final_logits.argmax(dim=1).tolist()

#     onehot_argmax = {v.values.argmax(): k for k, v in onehot.items()}
#     inv_map = {new_idx: onehot_argmax[orig_idx] for new_idx, orig_idx in enumerate(labels_idx)}
#     pred_labels = [inv_map[p] for p in preds]

#     correct = sum(pl == tl for pl, tl in zip(pred_labels, true_labels_list))
#     acc = correct / len(true_labels_list)

#     results.append((w1, w2, w3, acc))

#     if acc > best_acc:
#         best_acc = acc
#         best_w = (w1, w2, w3)

# # --- 3. Display results ---
# df_results = pd.DataFrame(results, columns=['w1', 'w2', 'w3', 'accuracy'])
# print(df_results)
# print(f"\n✅ Best Weights {best_w}, Accuracy {best_acc:.4f}")

# # --- 4. 3D Scatter Plot ---
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')
# sc = ax.scatter(df_results['w1'], df_results['w2'], df_results['accuracy'],
#                 c=df_results['accuracy'], cmap='viridis', s=40)
# ax.set_xlabel('w1')
# ax.set_ylabel('w2')
# ax.set_zlabel('Accuracy')
# ax.set_title('3D Scatter: Accuracy vs Weights (w3=1-w1-w2)')
# fig.colorbar(sc, ax=ax, label='Accuracy')
# plt.show()

# # --- 5. 2D Heatmap ---
# pivot = df_results.pivot(index='w1', columns='w2', values='accuracy')
# plt.figure(figsize=(10, 8))
# plt.imshow(pivot, origin='lower', aspect='auto', cmap='viridis')
# plt.xlabel('w2')
# plt.ylabel('w1')
# plt.title('Heatmap of Accuracy (w3=1-w1-w2)')
# plt.colorbar(label='Accuracy')
# plt.xticks(ticks=range(len(pivot.columns)), labels=[f"{x:.2f}" for x in pivot.columns], rotation=90)
# plt.yticks(ticks=range(len(pivot.index)), labels=[f"{y:.2f}" for y in pivot.index])
# plt.tight_layout()
# plt.show()


In [25]:
# len(predictions)

In [26]:
submission_df = pd.DataFrame({
    'annotation_id': ids,
    'concept_name': predictions
})


In [27]:
submission_df

,annotation_id,concept_name
0,1,Pandalus amplus
1,2,Funiculina
2,3,Funiculina
3,4,Funiculina
4,5,Funiculina
...,...,...
783,784,Lithodidae
784,785,Delectopecten
785,786,Scotoplanes globosa
786,787,Sebastolobus


In [28]:
# # Accuracy = number of correct predictions / total predictions
# correct = (submission_df['annotation_id'] == submission_df['concept_name']).sum()
# total = len(submission_df)

# accuracy = correct / total
# print("Accuracy:", accuracy)


In [29]:
submission_df.to_csv("submission.csv", index=False)

In [30]:
# import pandas as pd

# # Load the matrix
# pf = pd.read_csv('/kaggle/input/fathomnet-dataset/Taxonomic_metric (1).csv', index_col=0)

# total_score = 0.0
# count = 0
# for i in range(len(submission_df)):
#     true, predict = submission_df.iloc[i]
#     if true != predict:
#         total_score += pf.loc[true, predict]
#         count += 1
# score = total_score/len(submission_df)
# score_false = total_score/count
# print("Score: ", score)
# print("Score in false: ", score_false)